# Using **mem0** with Azure resources and Azure Identity credentials

This example demonstrates how to utilize **mem0** with Azure Identity credentials. 

**mem0** uses the following order of credentials when authenticating Azure OpenAI and Azure AI Search resources when an API is NOT provided. If a API key is provided it takes priority over Azure Identity credentials.

1. **Environment Credential:**
Azure client ID, secret, tenant ID, or certificate in environment variables for service principal authentication.

2. **Workload Identity Credential:**
Utilizes Azure Workload Identity (relevant for Kubernetes and Azure workloads).

3. **Managed Identity Credential:**
Authenticates as a Managed Identity (for apps/services hosted in Azure with Managed Identity enabled), this is the most secure production credential.

4. **Shared Token Cache Credential / Visual Studio Credential (Windows only):**
Uses cached credentials from Visual Studio sign-ins (and sometimes VS Code if SSO is enabled).

5. **Azure CLI Credential:**
Uses the currently logged-in user from the Azure CLI (`az login`), this is the most common development credential.

6. **Azure PowerShell Credential:**
Uses the identity from Azure PowerShell (`Connect-AzAccount`).

7. **Azure Developer CLI Credential:**
Uses the session from Azure Developer CLI (`azd auth login`).

## Prerequisites

### 1. Install Mem0

To use Mem0, install it using pip:

```bash
pip install "mem0ai"
```

### 2. Azure Credential setup

This example will utilize your Entra Id account for a local credential. The most command local credential used is the Azure CLI Credential (az login). The [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/?view=azure-cli-latest) must be installed on your development workstation for th is example.

### *Azure OpenAI setup*

#### Step A:  Enable Role-Based Access Control (RBAC) for Azure OpenAI

- In the Azure portal, go to your **Azure OpenAI** resource.
- Select **Access control (IAM)** in the left-hand menu. This is where you will manage role assignments for RBAC.

#### Step B: Assign Permissions to Your Entra ID Account

- Click **Add** > **Add role assignment**.
- For general read/write (data plane) access to Azure OpenAI, use:
    - **Cognitive Services OpenAI Contributor** – for full management (read/write/deploy models).
- On the **Members** tab, choose:
    - For your development account: select **User** and pick your Azure Entra ID account (the same used with `az login`).
- Click **Review + assign**.

### *Azure AI Search setup*

To allow your Azure Entra ID account to have read/write access to your Azure AI Search instance using Azure Identity, follow these steps:

#### Step C: Enable Role-Based Access Control (RBAC) for Azure AI Search

1. In the Azure Portal, navigate to your **Azure AI Search** service.
2. In the left menu, select **Settings** > **Keys**.
3. Change the authentication setting to **Role-based access control**, or to **Both** if you need API key compatibility. The default is “Key-based authentication”—you must switch it to use Azure roles.

#### Step D: Assign Permissions to Your Entra ID Account

1. **Go to Access Control (IAM):**
    - In the Azure Portal, select your Search service.
    - Click **Access Control (IAM)** on the left.
2. **Add a Role Assignment:**
    - Click **Add** > **Add role assignment**.
3. **Choose Role:**
    - For full data plane read/write access, select the **Search Service Contributor** role.
    - You can also use or clone existing built-in roles or create a custom role if more granularity is needed (e.g., specific indexes).
4. **Assign to User or Identity:**
    - Choose **User** and select your Entra ID account (the one used for `az login`).
5. **Complete the Assignment:**
    - Click **Review + Assign**.

#### Step E: Login to Azure

1. Install Azure CLI if not already installed, check your installation using the following command.

```bash
az --version
```

2. Open a command terminal issue the following command.

```bash
az login
```

Your default browser will open in order for you to login to Azure.  Once you login return to your terminal session.

3. Select your Azure Subscription if multiple are available.

You're now ready to execute the **mem0** using Azure resources and Azure Identity credentials.

## Imports and Constants


In [ ]:
import json
from mem0 import Memory

# Change these constants to your Azure OpenAI and Azure AI Search configurations
azure_openai_chat_model = "<your-azure-model>"  # e.g., "gpt-4o
azure_openai_endpoint = "<your-azure-endpoint>"  # e.g., "https://your-azure-openai-endpoint.openai.azure.com/"
azure_openai_chat_deployment = "<your-azure-deployment>"  # e.g., "gpt-4o"
azure_openai_chat_api_version = "<your-api-version>"  # e.g., "2024-12-01-preview"

azure_openai_embedding_model = "<your-embedding-model-name>"  # e.g., "text-embedding-3-large"
azure_openai_embedding_endpoint = (
    "<your-azure-endpoint>"  # e.g., "https://your-azure-openai-endpoint.openai.azure.com/"
)
azure_openai_embedding_deployment = "<your-embedding-deployment>"  # e.g., "text-embedding-3-large"
azure_openai_embedding_api_version = "<your-embedding-api-version>"  # e.g., "2024-02-01"

azure_search_service_name = "<your-azure-search-service-name>"  # e.g., "my-aiss" (name of the Azure AI Search resource)

# Create **mem0** configuration

1. LLM uses Azure OpenAI
2. Embedder uses Azure OpenAI
3. Vector Store uses Azure AI Search

In [ ]:
config = {
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": azure_openai_chat_model,
            "temperature": 0.1,
            "max_tokens": 2000,
            "azure_kwargs": {
                "azure_endpoint": azure_openai_endpoint,
                "azure_deployment": azure_openai_chat_deployment,
                "api_version": azure_openai_chat_api_version,
            },
        },
    },
    "embedder": {
        "provider": "azure_openai",
        "config": {
            "model": azure_openai_embedding_model,
            "azure_kwargs": {
                "azure_endpoint": azure_openai_embedding_endpoint,
                "azure_deployment": azure_openai_embedding_deployment,
                "api_version": azure_openai_embedding_api_version,
            },
        },
    },
    "vector_store": {
        "provider": "azure_ai_search",
        "config": {
            "service_name": azure_search_service_name,
            "collection_name": "mem0",
            "embedding_model_dims": 3072,
        },
    },
}

print("Configuration for Azure OpenAI and Azure AI Search.")
print(json.dumps(config, indent=2))

## Memory initialization  

In [ ]:
# use the configuration to create a Memory instance
m = Memory.from_config(config_dict=config)

## Store memories 

In [ ]:
# create a memory as a set of chat messages
messages = [
    {
        "role": "user",
        "content": "I'm planning to watch a movie tonight. Any recommendations?",
    },
    {
        "role": "assistant",
        "content": "How about a thriller movies? They can be quite engaging.",
    },
    {
        "role": "user",
        "content": "I'm not a big fan of thriller movies but I love sci-fi movies.",
    },
    {
        "role": "assistant",
        "content": "Got it! I'll avoid thriller recommendations and suggest sci-fi movies in the future.",
    },
]
# Store inferred memories (default behavior)
result = m.add(messages, user_id="alice", metadata={"category": "movie_recommendations"})

## Search memories

In [ ]:
# search stored memories
for result in m.search("what does alice love?", user_id="alice")["results"]:
    print(result["memory"], result["score"])